In [1]:
import numpy as np

from prism_pruner.conformer_ensemble import ConformerEnsemble
from prism_pruner.pruner import prune_by_moment_of_inertia, prune_by_rmsd

ensemble = ConformerEnsemble.from_xyz("../examples/ensemble.xyz")
ensemble.coords.shape

(1086, 136, 3)

In [2]:
%load_ext line_profiler

In [3]:
%%prun -s cumtime
prune_by_moment_of_inertia(
    ensemble.coords,
    ensemble.atoms,
    max_deviation=0.01,  # 1% difference
    debugfunction=print,
)

DEBUG: MOIPrunerConfig - k=50, rejected 449 (keeping 637/1086), in 0.1 s
DEBUG: MOIPrunerConfig - k=20, rejected 109 (keeping 528/1086), in 0.1 s
DEBUG: MOIPrunerConfig - k=10, rejected 27 (keeping 501/1086), in 0.1 s
DEBUG: MOIPrunerConfig - k=5, rejected 28 (keeping 473/1086), in 0.1 s
DEBUG: MOIPrunerConfig - k=2, rejected 38 (keeping 435/1086), in 0.3 s
DEBUG: MOIPrunerConfig - k=1, rejected 10 (keeping 425/1086), in 0.4 s
DEBUG: MOIPrunerConfig - keeping 425/1086 (1.0 s)
DEBUG: MOIPrunerConfig - Used cached data 105595/211707 times, 49.88% of total calls
 

         334293 function calls (334276 primitive calls) in 1.455 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.001    0.000    1.790    0.224 base_events.py:1966(_run_once)
        8    0.002    0.000    1.698    0.212 selectors.py:435(select)
        1    0.000    0.000    1.437    1.437 {built-in method builtins.exec}
     3660    0.554    0.000    0.982    0.000 pruner.py:166(_main_compute_subrow)
        8    0.033    0.004    0.933    0.117 {method 'poll' of 'select.epoll' objects}
       88    0.009    0.000    0.762    0.009 pruner.py:219(_main_compute_row)
        6    0.001    0.000    0.741    0.123 pruner.py:256(_main_compute_group)
   106112    0.383    0.000    0.383    0.000 pruner.py:153(evaluate_sim)
     1086    0.060    0.000    0.376    0.000 algebra.py:120(get_inertia_moments)
        1    0.004    0.004    0.365    0.365 pruner.py:142(__post_init__)
     1086    0.030    0.000    0.20

In [4]:
more_coords = np.concatenate((ensemble.coords, ensemble.coords))
more_coords.shape

(2172, 136, 3)

In [5]:
%%prun -s cumtime
prune_by_moment_of_inertia(
    more_coords,
    ensemble.atoms,
    max_deviation=0.01,  # 1% difference
    debugfunction=print,
)

DEBUG: MOIPrunerConfig - k=100, rejected 893 (keeping 1279/2172), in 0.2 s
DEBUG: MOIPrunerConfig - k=50, rejected 185 (keeping 1094/2172), in 0.1 s
DEBUG: MOIPrunerConfig - k=20, rejected 109 (keeping 985/2172), in 0.2 s
DEBUG: MOIPrunerConfig - k=10, rejected 45 (keeping 940/2172), in 0.3 s
DEBUG: MOIPrunerConfig - k=5, rejected 34 (keeping 906/2172), in 0.6 s
DEBUG: MOIPrunerConfig - k=2, rejected 58 (keeping 848/2172), in 1.3 s
DEBUG: MOIPrunerConfig - k=1, rejected 419 (keeping 429/2172), in 1.2 s
DEBUG: MOIPrunerConfig - keeping 429/2172 (4.1 s)
DEBUG: MOIPrunerConfig - Used cached data 371521/677646 times, 54.83% of total calls
 

         855617 function calls (855589 primitive calls) in 4.827 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       19    0.003    0.000    5.955    0.313 base_events.py:1966(_run_once)
       19    0.005    0.000    5.106    0.269 selectors.py:435(select)
      2/1    0.000    0.000    4.820    4.820 {built-in method builtins.exec}
     8224    2.412    0.000    3.856    0.000 pruner.py:166(_main_compute_subrow)
       19    0.118    0.006    3.791    0.200 {method 'poll' of 'select.epoll' objects}
      188    0.030    0.000    3.379    0.018 pruner.py:219(_main_compute_row)
        7    0.002    0.000    3.290    0.470 pruner.py:256(_main_compute_group)
   306125    1.293    0.000    1.293    0.000 pruner.py:153(evaluate_sim)
     2172    0.113    0.000    0.745    0.000 algebra.py:120(get_inertia_moments)
        1    0.008    0.008    0.702    0.702 pruner.py:142(__post_init__)
     2172    0.068    0.000    0.41

In [27]:
%%prun -s cumtime
pruned, mask = prune_by_rmsd(
    ensemble.coords[0:300],
    ensemble.atoms,
    max_rmsd=1.0,  # Will reject below 1 Å and
    debugfunction=print,
)

DEBUG: RMSDPrunerConfig - k=10, rejected 231 (keeping 69/300), in 0.4 s
DEBUG: RMSDPrunerConfig - k=2, rejected 35 (keeping 34/300), in 0.1 s
DEBUG: RMSDPrunerConfig - k=1, rejected 4 (keeping 30/300), in 0.1 s
DEBUG: RMSDPrunerConfig - keeping 30/300 (0.7 s)
DEBUG: RMSDPrunerConfig - Used cached data 539/2479 times, 21.74% of total calls
 

         195833 function calls (195826 primitive calls) in 0.664 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.657    0.657 {built-in method builtins.exec}
        1    0.000    0.000    0.657    0.657 <string>:1(<module>)
        1    0.000    0.000    0.656    0.656 pruner.py:374(prune_by_rmsd)
        1    0.000    0.000    0.656    0.656 pruner.py:290(prune)
        3    0.000    0.000    0.654    0.218 pruner.py:253(_main_compute_group)
      403    0.021    0.000    0.636    0.002 pruner.py:165(_main_compute_subrow)
     1940    0.066    0.000    0.619    0.000 pruner.py:113(evaluate_sim)
       13    0.004    0.000    0.581    0.045 pruner.py:217(_main_compute_row)
     1940    0.075    0.000    0.553    0.000 rmsd.py:9(rmsd_and_max)
     1940    0.043    0.000    0.274    0.000 algebra.py:151(get_alignment_matrix)
     1940    0.087    0.000    0.150    0.000 _linalg.py:1689(svd)